In [ ]:
import csv

In [ ]:
import os
from shutil import rmtree

# Delete the specs directory if it already exists
folder = SOURCE_MOUNT + '/specs'
if os.path.exists(folder):
    rmtree(folder)

# Get the text classification task spec files
!tao token_classification download_specs \
    -o $SPECS_DIR/text_classification \
    -r $RESULTS_DIR \
    2>&1|tee my_assessment/step1.txt # DO NOT REMOVE THIS LINE

In [ ]:
!ls $SOURCE_MOUNT/specs/text_classification

In [ ]:
!cat $SOURCE_MOUNT/specs/text_classification/train.yaml

In [ ]:
!tao text_classification dataset_convert \
    -e $DESTINATION_MOUNT/specs/text_classification/dataset_convert.yaml \
    -r $RESULTS_DIR/text_classification/dataset_convert \
    source_data_dir=$DATA_DIR/SST-2/\
    target_data_dir=$DATA_DIR/SST2/ \
    dataset_name=sst2

In [ ]:
# Open tsv and txt files(open txt file in write mode)
tsv_file = open("/dli/task/tao/data/SST2/train.tsv")
txt_file = open("/dli/task/tao/data/SST2/text_train.txt", "w")
txt_file_label = open("/dli/task/tao/data/SST2/labels_train.txt", "w")

# Read tsv file and use delimiter as \t. csv.reader
# function returns a iterator
# which is stored in read_csv
read_tsv = csv.reader(tsv_file, delimiter="\t")

# write data in txt file line by line
for row in read_tsv:
    txt_file.writelines(row[0]+'\n')
    txt_file_label.writelines(row[1]+'\n')

# close files
txt_file.close()
txt_file_label.close()
tsv_file.close()

In [ ]:
# Open tsv and txt files(open txt file in write mode)
tsv_file = open("/dli/task/tao/data/SST2/dev.tsv")
txt_file = open("/dli/task/tao/data/SST2/text_dev.txt", "w")
txt_file_label = open("/dli/task/tao/data/SST2/labels_dev.txt", "w")

# Read tsv file and use delimiter as \t. csv.reader
# function returns a iterator
# which is stored in read_csv
read_tsv = csv.reader(tsv_file, delimiter="\t")

# write data in txt file line by line
for row in read_tsv:
    txt_file.writelines(row[0]+'\n')
    txt_file_label.writelines(row[1]+'\n')

# close files
txt_file.close()
txt_file_label.close()
tsv_file.close()

In [ ]:
%%time
# For BERT training on SST-2:
!tao token_classification train \
    -e $SPECS_DIR/text_classification/train.yaml \
    -g 1  \
    -k $KEY \
    -r $RESULTS_DIR/sst2 \
    data_dir={DESTINATION_MOUNT}/data/SST2 \
    model.label_ids={DATA_DIR}/SST2/label_ids.csv \
    training_ds.text_file=$DATA_DIR/SST2/text_train.txt \
    training_ds.labels_file=$DATA_DIR/SST2/labels_train.txt \
    validation_ds.text_file=$DATA_DIR/SST2/text_dev.txt \
    validation_ds.labels_file=$DATA_DIR/SST2/labels_dev.txt \
    trainer.amp_level="O1" \
    trainer.precision=16 \
    trainer.max_epochs=2 \
    training_ds.num_samples=-1 \
    validation_ds.num_samples=-1 \
    2>&1|tee my_assessment/step2.txt # DO NOT REMOVE THIS LINE